# Elements Removal Visualization

In [11]:
import opstool as opst
import openseespy.opensees as ops

## Model

In [13]:
opst.load_ops_examples("Shell3D")

The original Tcl file comes from http://www.dinochen.com/, and the Python version is converted by opstool.tcl2py().


In [15]:
ModelData = opst.GetFEMdata(results_dir="opstool_output")
ModelData.get_model_data(save_file="ModelData.hdf5")
opsvis = opst.OpsVisPyvista(
    point_size=2,
    line_width=3,
    results_dir="opstool_output",
)
opsvis.model_vis(
    input_file="ModelData.hdf5",
    show_node_label=True,
    show_ele_label=True,
    show_outline=False,
)

Model data saved in opstool_output/ModelData.hdf5!

## Gravity analysis

In [4]:
ops.timeSeries("Linear", 1)
ops.pattern("Plain", 1, 1)
ops.load(14, 0.0, 0.0, -1000000.0, 0.0, 0.0, 0.0)
ops.load(20, 0.0, 0.0, -1000000.0, 0.0, 0.0, 0.0)
ops.load(26, 0.0, 0.0, -1000000.0, 0.0, 0.0, 0.0)
ops.load(32, 0.0, 0.0, -1000000.0, 0.0, 0.0, 0.0)
ops.load(56, 0.0, 0.0, -1000000.0, 0.0, 0.0, 0.0)
ops.load(72, 0.0, 0.0, -1000000.0, 0.0, 0.0, 0.0)
ops.load(88, 0.0, 0.0, -1000000.0, 0.0, 0.0, 0.0)
ops.load(104, 0.0, 0.0, -1000000.0, 0.0, 0.0, 0.0)
ops.load(120, 0.0, 0.0, -1000000.0, 0.0, 0.0, 0.0)
ops.constraints("Plain",)
ops.numberer("Plain",)
ops.system("BandGeneral")
ops.test("EnergyIncr", 1e-06, 200)
ops.algorithm("Newton",)
ops.integrator("LoadControl", 0.1)
ops.analysis("Static",)

In [ ]:
for i in range(10):
    ops.analyze(1)
    ModelData.get_resp_step(model_update=True)   # here!
ops.loadConst(0,)

## Elements removal

In [5]:
ops.wipeAnalysis()
ops.timeSeries("Linear", 2)
ops.pattern("Plain", 2, 2)
ops.constraints("Transformation",)
ops.test("NormDispIncr", 0.0001, 60)
ops.algorithm("KrylovNewton",)
ops.numberer("RCM",)
ops.system("BandGeneral",)
ops.integrator("LoadControl", 1)
ops.analysis("Static",)

In [6]:
eleTags = [32, 50, 52, 70, 31, 49, 51, 69]   # removed ele tags
for tag in eleTags:
    ops.remove("element", tag)
    if tag == 49:
        ops.fix(67, 1, 1, 1, 1, 1, 1)
    if tag == 51:
        ops.fix(81, 1, 1, 1, 1, 1, 1)
    if tag == 69:
        ops.fix(83, 1, 1, 1, 1, 1, 1)
    ok = ops.analyze(10)
    ModelData.get_resp_step(model_update=True)

In [7]:
ModelData.save_resp_all(save_file="RespStepData.hdf5", reset_state=True)

All responses data saved in opstool_output/RespStepData.hdf5!

Visual deformation, slider style

In [16]:
opsvis.deform_vis(
    input_file="RespStepData.hdf5",
    slider=True,
    response="disp",
    alpha=1.0,
    show_outline=False,
    show_origin=True,
    show_face_line=False,
    opacity=1,
    model_update=True,
)

create an animated gif